In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("C:/Users/tulas/Documents/Daksha/Datasets/Hospital_DataCleaned_Log (1).xlsx")

In [3]:
df["Hospital overall rating"].unique()

array([2, 1, 3, 4, 5], dtype=int64)

In [4]:
df.nunique()

Unnamed: 0                                                        447530
Score                                                               9155
Denominator                                                          864
Lower Estimate                                                       382
Higher Estimate                                                      671
HCAHPS Answer Percent                                                101
Number of Completed Surveys                                          119
Meets criteria for meaningful use of EHRs                              1
Hospital overall rating                                                5
Mortality national comparison                                          3
Safety of care national comparison                                     3
Readmission national comparison                                        3
Patient experience national comparison                                 3
Effectiveness of care national comparison          

In [5]:
df_test= df.drop(columns=["Footnote","Measure ID","Measure Name","HCAHPS Measure ID","HCAHPS Question","HCAHPS Answer Description","Survey Response Rate Percent","Hospital overall rating"],axis = 1)


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447530 entries, 0 to 447529
Data columns (total 32 columns):
Unnamed: 0                                                        447530 non-null int64
Score                                                             447530 non-null float64
Denominator                                                       447530 non-null float64
Lower Estimate                                                    447530 non-null float64
Higher Estimate                                                   447530 non-null float64
HCAHPS Answer Percent                                             447530 non-null float64
Number of Completed Surveys                                       447530 non-null float64
Meets criteria for meaningful use of EHRs                         447530 non-null int64
Mortality national comparison                                     447530 non-null int64
Safety of care national comparison                                447530 non-null int

# Stratified sampling 

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(df,df["Hospital overall rating"]):
    print("TRAIN:", train_index, "TEST:", test_index)
    train = df.loc[train_index]
    test = df.loc[test_index]

TRAIN: [413766 193394 334940 ... 342100 365500 377318] TEST: [201119 381122 261624 ...  85112 239124 152056]
TRAIN: [306319 186331 112399 ... 315406 385247  71501] TEST: [ 25085 372915 113836 ... 398747  42334 276234]
TRAIN: [190301 387689 216848 ... 249098 286475 398962] TEST: [172530 370604 161380 ... 262373 243022 325569]
TRAIN: [374225 233823 343295 ... 185251 386241 400086] TEST: [418141 115011 135140 ... 284902 107930 149674]
TRAIN: [386478 273179 446399 ... 253191 234167 157011] TEST: [238379 443663 393281 ... 411571   2308 391733]
TRAIN: [202049 261970  48191 ... 171053  15417 282338] TEST: [ 16338 378285 211293 ... 141559  19873  82542]
TRAIN: [241057  15821 330994 ...  45962  67398  34148] TEST: [339538   7641 121060 ... 371621  41641 350352]
TRAIN: [181219 437762 168658 ...  16043 259540 415746] TEST: [414181 153764 224837 ...  16764 240987  75598]
TRAIN: [301724  18480 400607 ...  62062 335029  36121] TEST: [198642 168730  57546 ... 128341   5330 278956]
TRAIN: [216294 1726

# Logistic Regression Model

In [8]:
from collections import Counter
from sklearn.datasets import make_classification
#Train data
X = train.drop(columns=["Footnote","Measure ID","Measure Name","HCAHPS Measure ID","HCAHPS Question","HCAHPS Answer Description","Survey Response Rate Percent","Hospital overall rating"],axis = 1)
Y = train["Hospital overall rating"]
print("Hospital overall rating unique values:")
print(sorted(Counter(Y).items()))
Y= Y.astype(float)

Hospital overall rating unique values:
[(1, 8093), (2, 22809), (3, 287493), (4, 31248), (5, 8381)]


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(train["Hospital overall rating"])

In [10]:
#X = train[["Denominator","Lower Estimate","Higher Estimate","HCAHPS Answer Percent","Number of Completed Surveys","Score"]]
#Y = train[["Hospital overall rating"]]

In [11]:
# Test data
test_X = test.drop(columns=["Footnote","Measure ID","Measure Name","HCAHPS Measure ID","HCAHPS Question","HCAHPS Answer Description","Survey Response Rate Percent","Hospital overall rating"],axis = 1)
test_Y = test[["Hospital overall rating"]]

# Model

In [12]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X,Y)
predictions = logmodel.predict(test_X)

C:\Users\tulas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\tulas\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


# Classification Report

In [16]:
from sklearn.matrics import classification_report
print(classification_report(test_Y,predictions))

ModuleNotFoundError: No module named 'sklearn.matrics'

# confusion matrix

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics
cm= metrics.confusion_matrix(test_Y,predictions)
print(cm)


In [ ]:
# import required modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
class_names=[1,2,3,4,5] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = pd.DataFrame()
X1 = train[["Denominator","Lower Estimate","Higher Estimate","HCAHPS Answer Percent","Number of Completed Surveys","Score"]]
vif["VIF Factor"] = [variance_inflation_factor(X1.values,i) for i in range(X1.shape[1])]
vif["features"]= X1.columns

In [ ]:
vif.round(1)

In [ ]:
X1 = train.drop(["Lower Estimate","Higher Estimate","Unnamed: 0","HCAHPS Measure ID,"],axis=1)
X1.iloc[0,:]

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics
print(classification_report(test_Y,predictions))
print("#########Coefficients of logistic regression model##################")
print("\n")
cm= metrics.confusion_matrix(test_Y,predictions)
print(cm)